In [1]:
import csv
''' Depois de tratar o texto, e guardar os csv's em (letras.csv), (nutricao.csv)
    Podemos de fato começar a aplicar o método do simplex para determinar as melhores dietas'''

nomes = ['HENRIQUE','LUCAS','RAFAEL'] #nossos nomes

#nutrientes da que nossa dieta precisa satisfazer 
nutrientes_minimos = ['Caloria','Calcio','VitaminaA','Riboflavina','AcidoAscorbico'] 

dieta_prodx_por_letra = dict() #dicionario de letra para dieta e prodx
with open('letras.csv',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dieta_prodx_por_letra[row['Letra']] = (float(row['Dieta']),float(row['ProdutoX']))

Vamos definir agora nosso problema de minimização!

Se gostaríamos de reduzir o custo, nossas variáveis serão todas $x_i$, onde $x_i$ significa quantos dólares gastamos no alimento $i$.
Como o médico nos passou quantidades mínimas para suprir de nutrientes, essas quantidades formarão nossa matriz de restrição.

Teremos algo do tipo:
$$
    x_1c_1 + x_2c_2 + ... + x_nc_n \geq c\\
    x_1p_1 + x_2p_2 + ... + x_np_n \geq p\\
    x_1v_1 + x_2v_2 + ... + x_nv_n \geq v\\
    x_1r_1 + x_2r_2 + ... + x_nr_n \geq r\\
    x_1a_1 + x_2a_2 + ... + x_na_n \geq a\\
$$

Onde $c_i$ é quanto de calórias o alimento $i$ dá por unidade de dolar gasto e $c$ é a quantidade mínima de calorias que o médico nos recomendou. (Fiz a mesma coisa para os outros nutrientes)

Como isso não está exatamente na forma padrão, teremos que multiplicar por -1 todas as linhas, de forma a tornar todo $\geq$ em $\leq$.

In [2]:
''' ideia 
M = [ [c1,c2,c3,...,cn],
      [p1,p2,p3,...,pn],
      ...
    ]
'''
alimentos = []
with open('nutricao.csv','r',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        alimentos.append(row)

Depois de conseguir resultados estranhos, percebi que as dimensões passadas pelo médico e pelas tabelas estão em dimensões diferentes:
- Dimensão Dieta: **$10^2$ calorias, $10^{-2}$ gramas, $10^2$ unidades internacionais, $10^{-1}$ miligramas e miligrama.**
- Dimensões Produto X: **$10^3$ calorias/dólar, $10^{−1}$ gramas/dólar, $10^3$ unidades internacionais/dólar, $10^{−1}$ miligramas/dólar, miligramas/dólar.**

In [3]:
matriz_sem_prod_x = []
for nutriente in nutrientes_minimos:
    linha_nutri = [float(x[nutriente]) for x in alimentos]
    matriz_sem_prod_x.append(linha_nutri)


In [4]:
#com a matriz de restrição sem o produto X pronta, já estamos quase finalizados, basta descobrir quanto produto X nos dá e o B para cada nome!
from copy import deepcopy
import numpy as np
import scipy.optimize

conversao_dieta = [10**-1, 10**-2, 10**-1, 10**-1, 1] # para conversão da restrição
conversao_prod_x = [1,10**-1,1,10**-1,1] # para conversao do produto x

def matriz(nome:str):
    nome = nome.upper()
    m = deepcopy(matriz_sem_prod_x)
    for i in range(5): #são cinco nutrientes
        if i < len(nome) and nome[i] in dieta_prodx_por_letra:
            letra = nome[i]
            val = float(dieta_prodx_por_letra[letra][1]) * conversao_prod_x[i]
            m[i].append(val)
        else: 
            m[i].append(0.)
    return np.matrix(m)

def restricoes(nome:str):
    nome = nome.upper()
    r = []
    for i in range(5):
        if i < len(nome) and nome[i] in dieta_prodx_por_letra:
            letra = nome[i]
            val = float(dieta_prodx_por_letra[letra][0]) * conversao_dieta[i]
            r.append(val)
        else:
            r.append(0.)
    return np.array(r)



def dieta(nome:str,method:str):
    m = -matriz(nome)
    r = -restricoes(nome)
    c = np.ones(len(alimentos) + 1) # esse é o funcional linear que será minizado
    return scipy.optimize.linprog(c=c,A_ub=m,b_ub=r,method=method)

In [5]:
matriz("henrique")

matrix([[4.470e+01, 3.600e+01, 8.400e+00, 2.060e+01, 7.400e+00,
         1.570e+01, 4.170e+01, 2.200e+00, 4.400e+00, 5.800e+00,
         2.400e+00, 2.600e+00, 5.800e+00, 1.430e+01, 1.100e+00,
         9.600e+00, 8.500e+00, 1.280e+01, 1.740e+01, 2.690e+01,
         6.900e+01],
        [2.000e+00, 1.700e+00, 1.510e+01, 6.000e-01, 1.640e+01,
         1.000e+00, 0.000e+00, 2.000e-01, 3.000e-01, 6.800e+00,
         3.700e+00, 4.000e+00, 3.800e+00, 1.800e+00, 0.000e+00,
         2.700e+00, 1.700e+00, 2.500e+00, 3.700e+00, 1.140e+01,
         8.200e+00],
        [0.000e+00, 3.090e+01, 2.600e+01, 5.580e+01, 2.810e+01,
         0.000e+00, 2.000e-01, 1.692e+02, 0.000e+00, 3.500e+00,
         6.900e+01, 7.200e+00, 1.660e+01, 6.700e+00, 9.184e+02,
         2.907e+02, 8.680e+01, 8.570e+01, 5.100e+00, 0.000e+00,
         9.100e+01],
        [3.330e+01, 7.900e+00, 2.350e+01, 0.000e+00, 1.030e+01,
         8.100e+00, 5.000e+00, 5.080e+01, 3.600e+00, 4.900e+00,
         5.800e+00, 4.500e+00, 5.900e+00,

In [6]:
print(restricoes("henrique"))

[ 3.    0.39  0.6   4.7  65.  ]


In [7]:
minha_dieta = dieta('henrique','highs')
vet = minha_dieta.x
vet.shape = (21,1)
M = matriz('henrique')

M*vet #vamos printar o valor nutricional da minha dieta (está satisfazendo o pedido!)

matrix([[ 3.        ],
        [ 0.39      ],
        [ 8.13040664],
        [ 4.7       ],
        [65.        ]])

In [8]:
minha_dieta

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 0.1281775971085443
              x: [[ 5.123e-02]
                  [ 0.000e+00]
                  ...
                  [ 2.178e-02]
                  [ 0.000e+00]]
            nit: 5
          lower:  residual: [ 5.123e-02  0.000e+00 ...  2.178e-02
                              0.000e+00]
                 marginals: [ 0.000e+00  5.273e-01 ...  0.000e+00
                              8.363e-02]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  7.530e+00  0.000e+00
                              0.000e+00]
                 marginals: [-7.768e-03 -3.112e-02 -0.000e+00 -1.773e-02

In [9]:
dieta('rafael','highs')

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 0.07963365018987803
              x: [ 7.056e-03  0.000e+00 ...  0.000e+00  6.521e-02]
            nit: 4
          lower:  residual: [ 7.056e-03  0.000e+00 ...  0.000e+00
                              6.521e-02]
                 marginals: [ 0.000e+00  4.149e-01 ...  3.423e-01
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  3.753e-01  0.000e+00  0.000e+00
                              0.000e+00]
                 marginals: [-1.323e-02 -0.000e+00 -3.816e-04 -1.226e-02
                             -1.691e-04]
 mip_node_count: 0
 mip_dual_bound: 0

In [10]:
dieta('lucas','highs')

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 0.15858707204585107
              x: [ 1.687e-02  0.000e+00 ...  0.000e+00  1.405e-01]
            nit: 3
          lower:  residual: [ 1.687e-02  0.000e+00 ...  0.000e+00
                              1.405e-01]
                 marginals: [ 0.000e+00  1.836e-01 ...  3.982e-01
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  1.736e-02  0.000e+00  7.524e-01
                              0.000e+00]
                 marginals: [-2.237e-02 -0.000e+00 -3.570e-04 -0.000e+00
                             -1.749e-04]
 mip_node_count: 0
 mip_dual_bound: 0